In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
import numpy as np
import sympy as sp
from IPython.display import HTML

plt.rcParams["animation.html"] = "jshtml"

In [ ]:
# Perform inverse kinematics for an in-plane mammal geometry leg.
# This will be used as a component of the full 3dof IK solution.

# This is conducted purely in 2d.  The geometry looks like:
#
#     O               +y
#    /                ^
#   /  <--- femur     |
#  /
#  ---------P        -> +x
#
#    ^tibia
#
# Where O is the origin and P is the end effector point in the 
# y/z coordinate frame.  +y is to the right in this diagram and
# +z is up.

# Px, Py - position of end effector
# l1, l2 - length of femur and tibia
# theta1 - angle from vertical of upper leg
# theta2 - angle from vertical of lower leg
Px, Py, l1, l2, theta1, theta2 = sp.symbols('Px Py l1 l2 theta1 theta2')

# This is a simple triangle.  We know the length of all three sides,
# which means we can find all three angles.

# Use the law of cosines to find the tibia angle first.  Note: A
# practical implementation of this needs to handle the limited domain
# of the relevant actuators and dimensions.
norm_PO_sq = Px * Px + Py * Py
norm_PO = sp.sqrt(norm_PO_sq)

cos_tibia_sub = l1 * l1 + l2 * l2 - norm_PO_sq

cos_tibia_inv = cos_tibia_sub / (2 * l1 * l2)
tibia_angle = sp.acos(cos_tibia_inv)

# Now do the femur.
cos_femur_sub = norm_PO_sq + l1 * l1 - l2 * l2
cos_femur_inv = cos_femur_sub / (2 * norm_PO * l1)
femur_angle = sp.acos(cos_femur_inv)

# Finally, convert these triangle angles into theta1 and theta2,
# which are both measured relative to vertical.
etheta1 = {theta1: sp.atan2(Py, Px) + femur_angle + sp.pi / 2}
ik = {
    'theta1': etheta1[theta1],
    'theta2': (theta1 - (sp.pi - tibia_angle)).subs(etheta1)
}
ik



In [ ]:
# Calculate the torque required from an in-plane mammal geometry leg
# to achieve a given end-effector force.  We assume that the 3rd (lateral) 
# degree of freedom is not a significant contributor to torque 
# requirements.

# Fx, Fy - force at end effector
# t1, t2 - torque on the upper and lower leg
# l1, l2 - length of upper and lower leg
# theta1 - angle from vertical of upper leg
# theta2 - angle from vertical of lower leg
#
# Reference free body diagram at: 
# https://docs.google.com/drawings/d/19iMuQCkOq_pnOxSgBEZ9HpjowesbO7r2yoPeQHNCyjA/edit?usp=sharing
Fx, Fy, t1, t2 = sp.symbols('Fx Fy t1 t2')
    
# We can trivially determine that F = I, so we'll just use F for I everywhere.
    
# First, generate the equations for the top member.
    
# Torque as measured from the top pin point.
top_torque = sp.Eq(t1 + t2 - sp.cos(theta1) * l1 * Fx - sp.sin(theta1) * l1 * Fy, 0)

# The bottom force equation is identical to the top, so provides no more 
# degrees of freedom.

# Torque as measured from the bottom pin point.
bottom_torque = sp.Eq(t2 - sp.cos(theta2) * l2 * Fx + sp.sin(theta2) * l2 * Fy, 0)

# Now we have 2 equations and two unknowns.
torque_equations = sp.solve([top_torque, bottom_torque], [t1, t2])
torque_equations

In [ ]:
%%capture

sl1 = 0.113  # femur length
sl2 = 0.119  # tibia length
height_off_ground = 0.04
ypos = -sl1 - sl2 + height_off_ground

def rotate(theta):
    c, s = math.cos(theta), math.sin(theta)
    return np.array(((c, -s), (s, c)))

def do_ik(x, y):
    sPx = x  
    sPy = y

    exp = [(l1, sl1), (l2, sl2), (Px, sPx), (Py, sPy)]
    current = {key : sp.N(value.subs(exp)) for key, value in ik.items()}

    # Now do the forward kinematics to find where the joint and end-effector 
    # position are.
    
    stheta1, stheta2 = current['theta1'], current['theta2']
    
    p1 = rotate(stheta1).dot(np.array([[0], [-sl1]]))
    p2 = p1 + rotate(stheta2).dot(np.array([[0], [-sl2]]))
    
    return {
        'stheta1': stheta1,
        'stheta2': stheta2,
        'p1': p1,
        'p2': p2,
    }
    
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(12, 4))
ax1.axis('equal')
ax1.set_xlim((-0.20, 0.2))
ax1.set_ylim((-0.30, 0.1))

xspace = np.linspace(-0.095, 0.085, 200)
def find_torques(xpos):
    r = do_ik(xpos, ypos)
    exp = [(theta1, r['stheta1']),
           (theta2, r['stheta2']),
           (l1, sl1), (l2, sl2),
           (Fx, 0), (Fy, 1)]
    torques = {key : sp.N(value.subs(exp)) for key, value in torque_equations.items()}
    return torques[t1], torques[t2]

torques = [find_torques(x) for x in xspace]

# plot the torques
ax2.plot(xspace, [x[0] for x in torques], label='femur')
ax2.plot(xspace, [x[1] for x in torques], label='tibia')
ax2.plot(xspace, [abs(x[0]) + abs(x[1]) for x in torques], label='total')
xvline = ax2.axvline(x=0)
ax2.legend()

# And finally plot the result.
leg_line, = ax1.plot([], [], 'ro-', lw=3, markersize=10)

def animate(i):
    r = do_ik(xspace[i], ypos)
    p1 = r['p1']
    p2 = r['p2']
    leg_line.set_data([0, p1[0], p2[0]], [0, p1[1], p2[1]])
    xvline.set_data([xspace[i], xspace[i]], [0, 1])
    return (leg_line,)

anim = animation.FuncAnimation(fig, animate, frames=len(xspace), interval=20)

In [ ]:
anim